## Loading the modules

In [14]:
# Loading the aiida profile.
import numpy as np
from aiida import load_profile, orm
from aiida.engine import submit, run_get_node
from aiida.plugins import DataFactory
from pymatgen.core import Structure
from pymatgen.io.cif import CifParser
from aiida_impuritysupercellconv.workflows.impuritysupercellconv import IsolatedImpurityWorkChain
from aiida_muon.workflows.find_muon import FindMuonWorkChain

load_profile()

Profile<uuid='130cdd60826f460f966c893aac81fd10' name='ifeanyi'>

## Setting the data

In [15]:
# StructureData and magmom.

py_struc = Structure.from_file("data/LaCoPO_bcs_file_26735.mcif", primitive=False)
aiida_structure = orm.StructureData(pymatgen=py_struc)
smag = aiida_structure.get_pymatgen_structure()
magmoms = py_struc.site_properties["magmom"]
magmom = orm.List([list(magmom) for magmom in magmoms])

#
codename = "pw_7.3.1@leonardo_core2"  # edit 
code = orm.load_code(codename)
#
pp_codename = "pp_7.3.1@leonardo_core2"  # edit
pp_code = orm.load_code(pp_codename)




# Metadata dictionary with a minimal computational settings.
metadata = {
    'options':{
        'resources' : {
            'num_machines': 1,
            'num_mpiprocs_per_machine': 112,
            },
        },
    'description': "FindMuonWorkChain via protocols"+ py_struc.formula,
    }

metadatapp = {
    'options':{
        'resources' : {
            'num_machines': 1,
            'num_mpiprocs_per_machine': 1,
            },
        },
    'description': "PP via protocols"+ py_struc.formula,
    }

metadatarlx = {
    'options':{
        'resources' : {
            'num_machines': 1,
            'num_mpiprocs_per_machine': 24,
            },
        },
    'description': "relax in isolatedimpurity via protocols"+ py_struc.formula,
    }


pw_settings = {#"ONLY_INITIALIZATION": False,
              "cmdline": ["-nk", "2", "-ndiag", "1"]}


/home/ifeanyi/anaconda3/envs/aiidadev/lib/python3.11/site-packages/pymatgen/io/cif.py:1225: UserWarning: Issues encountered while parsing CIF: Skipping relative stoichiometry check because CIF does not contain formula keys.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [ ]:
#README: BELOW IS THE LIST OF INPUT PARAMETERS THAT CAN BE DEFINED WITHIN THE BUILDER
"""
        pw_code,                                                                                           
        structure:                                       
        pp_code: orm.Code = None,                                    
        protocol: str =None,
        overrides: dict = {},             #prodide paramater dictionary to overide default settings     
        relax_unitcell: bool =False,      #if TRUE the input cell is first relaxed in the Isolatedimpurityworkchian  
        conv_thr: float = 0.0257,         # threshold for force convergence in the Isolatedimpurityworkchian
        magmom: list = None,              #List of 3D magnetic moments in Bohr magneton of the corresponding input structure, if not provided structure is assumed not magnetic
        options=None,
        sc_matrix: list =None,            #if Supercell matrix is not proovided the Isolatedimpurityworkchian is called to find it
        mu_spacing: float =1.0,           # Minimum distance in Angstrom between two starting muon positions  generated on a grid.
        kpoints_distance: float =0.301,   # The minimum desired distance in 1/Å between k-points in reciprocal space
        charge_supercell: bool =True,     #To run charged supercell for positive muon or not (neutral supercell) 
        hubbard: bool = True,             #To check and activate  DFT+U if Hubbard U value is found in the default dictionary
        spin_pol_dft: bool = True,        #Spin-polarised DFT simulation or not
        pseudo_family: str ="SSSP/1.2/PBE/efficiency",  #The label of the pseudo family
        **kwargs,
"""

## (1) Getting the builder from *get_builder_from_protocol()* function.


In [7]:
builder = FindMuonWorkChain.get_builder_from_protocol(
    pw_code=code,
    pp_code = pp_code,
    structure = aiida_structure,
    magmom = magmom,
    hubbard = True,
    pseudo_family = 'SSSP/1.3/PBE/efficiency',
    )

builder.pwscf.pw.metadata = metadata

builder.relax.base.pw.metadata = metadata

builder.pp_metadata = metadatapp

builder.pwscf.pw.settings = orm.Dict(pw_settings)  
builder.relax.base.pw.settings = orm.Dict(pw_settings)
#builder.qe_settings = orm.Dict(pw_settings)  #also works


"""Here, since the sc_matrix is not provided in the input, the workchain
automatically calls the IsolatedImpurityWorkChain to find the converged supercell.
Hence, the metadata for the calculations are to be defined."""

#bulder.impuritysupercellconv_metadata = metadata #also works

if 'impuritysupercellconv' in builder: # i.e. sc_matrix = None
    builder.impuritysupercellconv.pwscf.pw.metadata = metadata
    builder.impuritysupercellconv.pwscf.pw.settings = orm.Dict(pw_settings)
    if 'relax' in builder.impuritysupercellconv: # i.e. relax_musconv = True
        builder.impuritysupercellconv.relax.base.pw.metadata = metadatarlx
        builder.impuritysupercellconv.relax.base.pw.settings = orm.Dict(pw_settings)

In [8]:
#UNCOMMENT below TO run the calculations

#submission = submit(builder)
#submission.pk

28394

# (2) Some lines to query and get the results after the calculations

In [19]:
#READ RESULTS

node=orm.load_node(28394) #findmuonworkchain

#outputs and inputs
uuid_dict=node.outputs.all_index_uuid.get_dict()
allsites_dict=node.outputs.all_sites.get_dict()
uniquesites_dict=node.outputs.unique_sites.get_dict()
input_strc = node.inputs.structure.get_pymatgen_structure()
input_str=input_strc.copy()

#sc_matrix= node.inputs.sc_matrix

node4=orm.load_node(28395) #isolated impurity workchain
sc_mat_array =node4.outputs.Converged_SCmatrix.get_array('sc_mat')
sc_matrix = sc_mat_array.tolist()
print("The supercell-Matrix is"+str(sc_matrix[:]))


The supercell-Matrix is[[3, 0, 0], [0, 3, 0], [0, 0, 2]]


In [21]:
#PRINT MAGNETIC SYMMETRY UNIQUE SITES

e_min=np.min([uniquesites_dict[key][1] for key in uniquesites_dict])
#fo=open('unique_energy_list.txt','w')              #uncomment to write to files
#fo.write("# site-label        energy(eV)      energy-diff(eV). The supercell-Matrix is"+str(sc_matrix[:])+"\n")
print("# site-label energy-diff(eV).")

for key in uniquesites_dict:
    print("H"+key, (uniquesites_dict[key][1]-e_min))
    #fo.write("%s %16f %16f \n "%  ("H"+key, uniquesites_dict[key][1], (uniquesites_dict[key][1]-e_min)))
    py_struc = Structure.from_dict(uniquesites_dict[key][0])
    musite = py_struc.frac_coords[py_struc.atomic_numbers.index(1)]
    mupos = np.dot(musite,sc_matrix)%1
    input_str.append(species = "H"+key, coords = mupos, coords_are_cartesian = False, validate_proximity = False)
    #py_struc.to(filename="H"+key+".cif".format())           #uncomment to write to cif
#fo.close()
#input_str.to(filename="unique_mu_in_inputcell.cif".format())  #uncomment to write to cif

# site-label energy-diff(eV).
H1 0.4817930600256659
H3 0.0
H4 0.3722669699927792
H8 0.19405580998864025
H10 0.37221458001295105
H11 0.19404954998753965
H13 0.4892155000125058
H17 0.37230356002692133
H19 0.19402995001291856


In [16]:
# PRINT FIELD CONTRIBUTION ONLY IF MAGNETIC

unique_hf_dict=node.outputs.unique_sites_hyperfine.get_dict()
unique_dip_list=node.outputs.unique_sites_dipolar

print('label, B_hf (T), Bdip_vec (T), |B_tot (T)| = Bdip+ B_hf')
for key in unique_hf_dict:
    for i in range(len(unique_dip_list)):
        if str(unique_dip_list[i]['idx'])== key:
            Bdip= unique_dip_list[i]['Bdip']
            Btot=unique_dip_list[i]['B_T_norm']
    print("H"+key, unique_hf_dict[key][1], Bdip,Btot)

label, B_hf (T), Bdip_vec (T), |B_tot (T)| = Bdip+ B_hf
H1 0.001064440986002 [0.00023819848011351, 0.00023819848011351, 0.052853729488549] 0.053919222772168
H3 -0.2099416114742 [-1.0479204921601e-06, -1.047920492166e-06, 0.044366871457889] 0.16557474002294
H4 -0.4817353080231 [0.018312608852916, 0.018312608852916, -0.049718087441246] 0.53208402986425
H8 -0.4718784020351 [-2.2018730625083e-06, -0.0020817400694329, 0.16350942918333] 0.30837599949448
H10 -0.4818821130059 [-0.018363510765676, 0.018547175529862, -0.050083140830619] 0.53260515162466
H11 -0.4724603789312 [0.00060036484675582, -6.9743273577443e-05, 0.16353330769812] 0.30892766247573
H13 -0.002484307321433 [0.00039694354726544, 2.4838969014797e-07, 0.053241464408579] 0.05075870920158
H17 -0.482464089902 [-0.018164700858855, -0.018164700858854, -0.049664801149795] 0.53274859869041
H19 -0.4722768727027 [-0.00066496764291781, -3.0311164204779e-07, 0.16350731404185] 0.30877027469897


In [22]:
#PRINT ALL THe SITES

input_str=input_strc.copy()

#fo=open('all_energy_list.txt','w') #uncomment to write to file
#fo.write("# site-label        energy(eV)      energy-diff(eV). The supercell-Matrix is"+str(sc_matrix[:])+"\n")
for key in allsites_dict:
    print("H"+key, (allsites_dict[key][1]-e_min))
    #fo.write("%s %16f %16f \n "%  ("H"+key, allsites_dict[key][1], (allsites_dict[key][1]-e_min)))
    py_struc = Structure.from_dict(allsites_dict[key][0])
    musite = py_struc.frac_coords[py_struc.atomic_numbers.index(1)]
    mupos = np.dot(musite,sc_matrix)%1
    input_str.append(species = "H"+key, coords = mupos, coords_are_cartesian = False, validate_proximity = False)
    #py_struc.to(filename="H"+key+".cif".format()) #uncomment to write to cif

#fo.close()
#input_str.to(filename="all_converged_sites_in_inputcell.cif".format()) #uncomment to write to cif

H1 0.4817930600256659
H2 0.4817627199809067
H3 0.0
H4 0.3722669699927792
H5 0.37224342999979854
H6 0.489223119977396
H7 -1.360021997243166e-06
H8 0.19405580998864025
H9 0.1940408400259912
H10 0.37221458001295105
H11 0.19404954998753965
H12 0.19402464997256175
H13 0.4892155000125058
H14 3.809982445091009e-06
H15 1.3330020010471344e-05
H16 2.0400038920342922e-06
H17 0.37230356002692133
H18 0.3722645199741237
H19 0.19402995001291856
H20 0.19402941002044827
